# Модель нейронной сети
##  Автор: Луттиев Владислав, Учебная группа: о.ИЗДтс 23.2/Б3-22


In [1]:
# Импорт библиотек

import os
import random
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

# PyTorch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

# Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix,
    classification_report,
)

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Загрузка набора данных

paths = {
    'train': 'nn-dataset/leads_train_nn.csv',
    'val': 'nn-dataset/leads_val_nn.csv',
    'test': 'nn-dataset/leads_test_nn.csv',
}

# Проверка существования файлов
missing = [p for p in paths.values() if not os.path.exists(p)]
if missing:
    raise FileNotFoundError(f'Не найден {missing}.')

train_df = pd.read_csv(paths['train'])
val_df = pd.read_csv(paths['val'])
test_df = pd.read_csv(paths['test'])

# Проверка целевой переменной
target_col = 'Converted'
feature_cols = [c for c in train_df.columns if c != target_col]

auth_cols = set(train_df.columns)
for name, df in [('val', val_df), ('test', test_df)]:
    if set(df.columns) != auth_cols:
        diff = auth_cols ^ set(df.columns)
        raise ValueError(f'Несоответствие столбцов для {name}: {diff}')

print('Train:', train_df.shape, 'Val:', val_df.shape, 'Test:', test_df.shape)


Train: (5383, 172) Val: (1154, 172) Test: (1154, 172)


In [3]:
# Фиксируем генераторы случайных чисел

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [4]:
# Устройство нейронной сети
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


In [5]:
# Подготовка и масштабирование признаков

num_cols = [c for c in feature_cols if c.startswith('num__')]
scaler = StandardScaler()

# Масштабирование
if num_cols:
    train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
    val_df[num_cols] = scaler.transform(val_df[num_cols])
    test_df[num_cols] = scaler.transform(test_df[num_cols])

# Преобразование
def to_arrays(df):
    X = df[feature_cols].to_numpy(dtype=np.float32)
    y = df[target_col].astype(np.float32).to_numpy()
    return X, y


X_train, y_train = to_arrays(train_df)
X_val, y_val = to_arrays(val_df)
X_test, y_test = to_arrays(test_df)

print('Размер входного слоя нейронной сети:', X_train.shape[1], '| Масштабированных числовых признаков:', len(num_cols))


Размер входного слоя нейронной сети: 171 | Масштабированных числовых признаков: 6


In [6]:
# Создание набора данных
class LeadDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]

batch_size = 256
# Создание
train_ds = LeadDataset(X_train, y_train)
val_ds   = LeadDataset(X_val, y_val)
test_ds  = LeadDataset(X_test, y_test)

# Зазрузчик
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)


In [7]:
# Создание примерной модели нейронной сети, на вход подается 

class LeadMLP(nn.Module):
    def __init__(self, input_layer):
        super().__init__() # Инициализация базового класса nn.Module
        
        # Архитектура сети
        self.net = nn.Sequential(
            nn.Linear(input_layer, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze(1)


In [8]:
# Инициализируем модель, функции потерь и оптимизатора
model = LeadMLP(X_train.shape[1]).to(device)
# Расчёт веса положительного класса
pos_weight_value = (len(y_train) - y_train.sum()) / y_train.sum()
# Функция потерь
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight_value], device=device))
# Оптимизатор
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [9]:
# Cтруктура модели
print("Model Summary...\n")
summary(model, input_size=(X_train.shape[1],))

Model Summary...

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]          44,032
       BatchNorm1d-2                  [-1, 256]             512
              ReLU-3                  [-1, 256]               0
           Dropout-4                  [-1, 256]               0
            Linear-5                  [-1, 128]          32,896
       BatchNorm1d-6                  [-1, 128]             256
              ReLU-7                  [-1, 128]               0
           Dropout-8                  [-1, 128]               0
            Linear-9                   [-1, 64]           8,256
             ReLU-10                   [-1, 64]               0
          Dropout-11                   [-1, 64]               0
           Linear-12                    [-1, 1]              65
Total params: 86,017
Trainable params: 86,017
Non-trainable params: 0
---------------